# Importing Modules and Checking Directory 

In [36]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [37]:
pd.__version__

'1.1.5'

In [38]:
os.getcwd()


'/Users/Livi/Projects/Black_mental_health/notebooks/Data_wrangling'

# Uploading Data 

In [91]:
med = pd.read_csv("../../data/processed/final2012MEDICARE.csv", index_col=0)

In [92]:
med.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893864 entries, 0 to 919020
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   npi                               893864 non-null  int64  
 1   entity_code                       893864 non-null  object 
 2   full_zip                          893864 non-null  int64  
 3   state                             893864 non-null  object 
 4   country                           893864 non-null  object 
 5   provider_type                     893864 non-null  object 
 6   medicare_participation_indicator  893864 non-null  object 
 7   number_of_hcpcs                   893864 non-null  int64  
 8   number_of_services                893864 non-null  int64  
 9   total_beneficiaries               893864 non-null  int64  
 10  total_submitted_charges           893864 non-null  float64
 11  total_allowed_payment             893864 non-null  f

In [93]:
med.full_zip = med.full_zip.apply(lambda x: str(x).zfill(5))
med.npi = med.npi.astype('str')

In [94]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

In [80]:
census = pd.read_csv('../../data/processed/final2012CENSUS.csv', index_col=0)

In [84]:
census.head()

,geo_id,nonfamily_households,family_households,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,...,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,pop_15_and_over,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,do_date
0,79916,0,55,19,21,15,0,0,0,0,...,2250,406,65,2802,2119,532,40,0,111,2012
1,61112,82,0,33,34,0,0,0,0,0,...,82,0,0,82,0,0,0,71,11,2012
2,36112,0,16,0,3,0,0,3,10,0,...,313,24,0,330,283,43,4,0,0,2012
3,98235,0,30,0,21,0,0,0,0,0,...,30,13,0,64,26,38,0,0,0,2012
4,70747,27,13,0,0,8,0,0,0,0,...,71,0,0,44,16,9,0,11,8,2012


In [82]:
census.reset_index(drop=True, inplace=True)

In [83]:
census.do_date = pd.to_datetime(census.do_date).dt.year

In [85]:
info = pd.DataFrame(index=census.columns.values,
                    data={'data type': census.dtypes, \
                          'non-null values': census.notnull().sum(),
                          'unique values': census.nunique(), \
                          'null values': census.isnull().sum()})
info.head(10)

,data type,non-null values,unique values,null values
geo_id,int64,19275,19275,0
nonfamily_households,int64,19275,5782,0
family_households,int64,19275,8441,0
rent_burden_not_computed,int64,19275,893,0
rent_over_50_percent,int64,19275,2620,0
rent_40_to_50_percent,int64,19275,1218,0
rent_35_to_40_percent,int64,19275,964,0
rent_30_to_35_percent,int64,19275,1186,0
rent_25_to_30_percent,int64,19275,1383,0
rent_20_to_25_percent,int64,19275,1461,0


In [86]:
census.geo_id = census.geo_id.astype('str').apply(lambda x: x.zfill(5))

In [87]:
#checkpoint 1
checkpoint1a = census.copy()
checkpoint1b = med.copy()

# For modeling purposes, I want to aggregate the Med data by zip code before I merge with Census. 

In [95]:
med.columns

Index(['npi', 'entity_code', 'full_zip', 'state', 'country', 'provider_type',
       'medicare_participation_indicator', 'number_of_hcpcs',
       'number_of_services', 'total_beneficiaries', 'total_submitted_charges',
       'total_allowed_payment', 'total_medicare_payment',
       'avg_beneficiary_age', 'avg_hcc_risk_score', 'services_per_capita',
       'allowed_payment_per_capita', 'medicare_payment_per_capita',
       'submitted_charges_per_capita', 'region', 'subregion',
       'doctor_category'],
      dtype='object')

#### I want to binarize the object columns first, except region, subregion, and state (because that will be clear by the zip code). 

In [96]:
med.medicare_participation_indicator = med.medicare_participation_indicator.replace('Y',1).replace('N',0)

In [97]:
med = pd.get_dummies(med, columns = ['doctor_category', 'provider_type', 'entity_code'])

In [98]:
med.columns = med.columns.str.replace(' ', '_')

In [99]:
list(med.columns)

['npi',
 'full_zip',
 'state',
 'country',
 'medicare_participation_indicator',
 'number_of_hcpcs',
 'number_of_services',
 'total_beneficiaries',
 'total_submitted_charges',
 'total_allowed_payment',
 'total_medicare_payment',
 'avg_beneficiary_age',
 'avg_hcc_risk_score',
 'services_per_capita',
 'allowed_payment_per_capita',
 'medicare_payment_per_capita',
 'submitted_charges_per_capita',
 'region',
 'subregion',
 'doctor_category_General_Medicine_Pracitioner',
 'doctor_category_Medical_Specialists',
 'doctor_category_Mental_Health_Providers',
 'doctor_category_Other_Health_Services',
 'doctor_category_Other_Specialists',
 'doctor_category_Surgery_Specialists',
 'provider_type_Addiction_Medicine',
 'provider_type_All_Other_Suppliers',
 'provider_type_Allergy/Immunology',
 'provider_type_Ambulance_Service_Supplier',
 'provider_type_Ambulatory_Surgical_Center',
 'provider_type_Anesthesiologist_Assistants',
 'provider_type_Anesthesiology',
 'provider_type_Audiologist_(billing_independe

In [100]:
grouping = med.groupby('full_zip').agg(
    {'npi':'count',
 'state':lambda x:x.value_counts().index[0],
 'number_of_services':'sum',
 'total_beneficiaries':'sum',
 'total_submitted_charges':'sum',
 'total_allowed_payment':'sum',
 'total_medicare_payment':'sum',
 'avg_beneficiary_age':'mean',
 'avg_hcc_risk_score':'mean',
 'region':lambda x:x.value_counts().index[0],
 'subregion':lambda x:x.value_counts().index[0],
 'doctor_category_General_Medicine_Pracitioner':'sum',
 'doctor_category_Medical_Specialists':'sum',
 'doctor_category_Mental_Health_Providers':'sum',
 'doctor_category_Other_Health_Services':'sum',
 'doctor_category_Other_Specialists':'sum',
 'doctor_category_Surgery_Specialists':'sum',
 'provider_type_Addiction_Medicine':'sum',
 'provider_type_All_Other_Suppliers':'sum',
 'provider_type_Allergy/Immunology':'sum',
 'provider_type_Ambulance_Service_Supplier':'sum',
 'provider_type_Ambulatory_Surgical_Center':'sum',
 'provider_type_Anesthesiologist_Assistants':'sum',
 'provider_type_Anesthesiology':'sum',
 'provider_type_Audiologist_(billing_independently)':'sum',
 'provider_type_CRNA':'sum',
 'provider_type_Cardiac_Electrophysiology':'sum',
 'provider_type_Cardiac_Surgery':'sum',
 'provider_type_Cardiology':'sum',
 'provider_type_Certified_Clinical_Nurse_Specialist':'sum',
 'provider_type_Certified_Nurse_Midwife':'sum',
 'provider_type_Chiropractic':'sum',
 'provider_type_Clinical_Laboratory':'sum',
 'provider_type_Clinical_Psychologist':'sum',
 'provider_type_Colorectal_Surgery_(formerly_proctology)':'sum',
 'provider_type_Critical_Care_(Intensivists)':'sum',
 'provider_type_Dermatology':'sum',
 'provider_type_Diagnostic_Radiology':'sum',
 'provider_type_Emergency_Medicine':'sum',
 'provider_type_Endocrinology':'sum',
 'provider_type_Family_Practice':'sum',
 'provider_type_Gastroenterology':'sum',
 'provider_type_General_Practice':'sum',
 'provider_type_General_Surgery':'sum',
 'provider_type_Geriatric_Medicine':'sum',
 'provider_type_Geriatric_Psychiatry':'sum',
 'provider_type_Gynecological/Oncology':'sum',
 'provider_type_Hand_Surgery':'sum',
 'provider_type_Hematology':'sum',
 'provider_type_Hematology/Oncology':'sum',
 'provider_type_Hospice_and_Palliative_Care':'sum',
 'provider_type_Independent_Diagnostic_Testing_Facility':'sum',
 'provider_type_Infectious_Disease':'sum',
 'provider_type_Internal_Medicine':'sum',
 'provider_type_Interventional_Pain_Management':'sum',
 'provider_type_Interventional_Radiology':'sum',
 'provider_type_Licensed_Clinical_Social_Worker':'sum',
 'provider_type_Mammographic_Screening_Center':'sum',
 'provider_type_Mass_Immunization_Roster_Biller':'sum',
 'provider_type_Maxillofacial_Surgery':'sum',
 'provider_type_Medical_Oncology':'sum',
 'provider_type_Multispecialty_Clinic/Group_Practice':'sum',
 'provider_type_Nephrology':'sum',
 'provider_type_Neurology':'sum',
 'provider_type_Neuropsychiatry':'sum',
 'provider_type_Neurosurgery':'sum',
 'provider_type_Nuclear_Medicine':'sum',
 'provider_type_Nurse_Practitioner':'sum',
 'provider_type_Obstetrics/Gynecology':'sum',
 'provider_type_Occupational_therapist':'sum',
 'provider_type_Ophthalmology':'sum',
 'provider_type_Optometry':'sum',
 'provider_type_Oral_Surgery_(dentists_only)':'sum',
 'provider_type_Orthopedic_Surgery':'sum',
 'provider_type_Osteopathic_Manipulative_Medicine':'sum',
 'provider_type_Otolaryngology':'sum',
 'provider_type_Pain_Management':'sum',
 'provider_type_Pathology':'sum',
 'provider_type_Pediatric_Medicine':'sum',
 'provider_type_Peripheral_Vascular_Disease':'sum',
 'provider_type_Pharmacy':'sum',
 'provider_type_Physical_Medicine_and_Rehabilitation':'sum',
 'provider_type_Physical_Therapist':'sum',
 'provider_type_Physician_Assistant':'sum',
 'provider_type_Plastic_and_Reconstructive_Surgery':'sum',
 'provider_type_Podiatry':'sum',
 'provider_type_Portable_X-ray':'sum',
 'provider_type_Preventive_Medicine':'sum',
 'provider_type_Psychiatry':'sum',
 'provider_type_Psychologist_(billing_independently)':'sum',
 'provider_type_Public_Health_Welfare_Agency':'sum',
 'provider_type_Pulmonary_Disease':'sum',
 'provider_type_Radiation_Oncology':'sum',
 'provider_type_Radiation_Therapy':'sum',
 'provider_type_Registered_Dietician/Nutrition_Professional':'sum',
 'provider_type_Rheumatology':'sum',
 'provider_type_Slide_Preparation_Facility':'sum',
 'provider_type_Speech_Language_Pathologist':'sum',
 'provider_type_Sports_Medicine':'sum',
 'provider_type_Surgical_Oncology':'sum',
 'provider_type_Thoracic_Surgery':'sum',
 'provider_type_Unknown_Physician_Specialty_Code':'sum',
 'provider_type_Unknown_Supplier/Provider':'sum',
 'provider_type_Urology':'sum',
 'provider_type_Vascular_Surgery':'sum',
 'medicare_payment_per_capita':'mean',
 'submitted_charges_per_capita': 'mean', 
 'allowed_payment_per_capita': 'mean',
 'services_per_capita': 'mean',
    'entity_code_I':'sum',
 'entity_code_O':'sum'})




In [101]:
merged = grouping.merge(census, left_on='full_zip', right_on='geo_id', how='left')


In [102]:
merged.columns[merged.columns.str.contains('pop')]

Index(['total_pop', 'male_pop', 'female_pop', 'white_pop', 'black_pop',
       'asian_pop', 'hispanic_pop', 'amerindian_pop', 'other_race_pop',
       'two_or_more_races_pop', 'not_hispanic_pop', 'employed_pop',
       'unemployed_pop', 'pop_16_over', 'pop_in_labor_force',
       'not_us_citizen_pop', 'pop_25_64', 'pop_determined_poverty_status',
       'population_1_year_and_over', 'population_3_years_over',
       'pop_25_years_over', 'pop_5_years_over', 'pop_15_and_over',
       'pop_never_married', 'pop_now_married', 'pop_separated', 'pop_widowed',
       'pop_divorced'],
      dtype='object')

In [103]:
merged['percent_black_pop'] = merged.black_pop / merged.total_pop
merged['percent_white_pop'] = merged.white_pop / merged.total_pop
merged['percent_amerindian_pop'] = merged.amerindian_pop / merged.total_pop
merged['percent_asian_pop'] = merged.asian_pop / merged.total_pop
merged['percent_other_race_pop'] = merged.other_race_pop / merged.total_pop
merged['percent_two_or_more_races_pop'] = merged.two_or_more_races_pop / merged.total_pop
merged['percent_hispanic_pop'] = merged.hispanic_pop / merged.total_pop

In [106]:
merged.head()

,npi,state,number_of_services,total_beneficiaries,total_submitted_charges,total_allowed_payment,total_medicare_payment,avg_beneficiary_age,avg_hcc_risk_score,region,...,pop_widowed,pop_divorced,do_date,percent_black_pop,percent_white_pop,percent_amerindian_pop,percent_asian_pop,percent_other_race_pop,percent_two_or_more_races_pop,percent_hispanic_pop
0,38,MA,50001,8303,4780781.13,2758545.46,2097877.44,70.052632,1.350534,Northeast,...,1131,1847,2012,0.014557,0.906847,0.000863,0.017722,0.004948,0.008400,0.046663
1,57,MA,83392,15550,8905593.58,4565729.49,3481999.66,66.508772,1.096372,Northeast,...,555,1607,2012,0.049481,0.732015,0.002542,0.106310,0.001288,0.026534,0.081830
2,3,MA,579,216,96392.66,50287.66,35960.56,67.000000,0.936333,Northeast,...,3,1,2012,0.046606,0.780968,0.000000,0.104643,0.002570,0.010101,0.054581
3,15,MA,9038,2294,1565452.70,489756.75,361674.90,65.266667,1.299500,Northeast,...,183,486,2012,0.000000,0.972461,0.001758,0.013672,0.000000,0.000000,0.012109
4,25,MA,24147,4719,2590717.26,1300648.29,990671.49,69.280000,1.148276,Northeast,...,519,796,2012,0.012746,0.921264,0.000000,0.016446,0.000891,0.020489,0.028164


In [108]:
merged.rename(columns = {
    'medicare_payment_per_capita':'avg_medicare_payment_per_capita',
 'submitted_charges_per_capita': 'avg_submitted_charges_per_capita', 
 'allowed_payment_per_capita': 'avg_allowed_payment_per_capita',
 'services_per_capita': 'avg_number_services_per_capita',
'npi':'number_of_providers',
}, inplace=True)

In [110]:
#saving census medicare merged
merged.to_csv('../../data/processed/final2012aggregated.csv')
